# **Chapter 6: Test Design Techniques**

---

## **6.1 Introduction to Test Design**

Test design is the process of creating test cases based on test conditions, using systematic techniques to ensure comprehensive coverage while minimizing redundancy. Effective test design transforms testing from random guesswork into a systematic engineering activity.

**Why Test Design Matters:**
- **Efficiency:** Well-designed tests find more bugs with fewer cases
- **Coverage:** Ensures all important scenarios are tested
- **Repeatability:** Standardized techniques produce consistent results
- **Traceability:** Links test cases back to requirements or code structure
- **Communication:** Common techniques enable team collaboration

**Test Design Process:**
```
Test Conditions (What to test)
        ↓
Test Design Techniques (How to derive cases)
        ↓
Test Cases (Specific inputs, actions, expected results)
        ↓
Test Procedures (Steps to execute)
```

---

## **6.2 Black Box Testing Techniques**

Black box (specification-based) techniques derive test cases from requirements, specifications, and user stories without knowledge of the internal code structure. These align with **ISTQB Foundation Level** syllabus and are the primary techniques for system and acceptance testing.

### **6.2.1 Equivalence Partitioning (EP)**

**Concept:** Divide input data into classes (partitions) where the system behaves equivalently. Testing one value from each partition is assumed equivalent to testing all values in that class.

**Theory:**
- Valid partitions: Acceptable inputs
- Invalid partitions: Unacceptable inputs (error handling)
- Each partition should be tested at least once

**Mathematical Basis:**
If domain $D$ is partitioned into subsets $P_1, P_2, ..., P_n$ such that:
- $\bigcup_{i=1}^{n} P_i = D$ (Complete coverage)
- $P_i \cap P_j = \emptyset$ for $i \neq j$ (Disjoint partitions)

Then selecting test case $t_i \in P_i$ represents all values in $P_i$.

```python
class EquivalencePartitioning:
    """
    Implementing Equivalence Partitioning for test design
    """
    
    def analyze_age_validation(self):
        """
        Example: System accepts ages 18-65 for registration
        """
        partitions = {
            "valid_partitions": [
                {
                    "id": "EP1",
                    "range": "18-65",
                    "description": "Valid adult age",
                    "representative_values": [18, 30, 65],  # Boundaries + middle
                    "expected_result": "Accept registration"
                }
            ],
            "invalid_partitions": [
                {
                    "id": "EP2",
                    "range": "< 18",
                    "description": "Too young (minor)",
                    "representative_values": [0, 17],  # Boundary + extreme
                    "expected_result": "Reject with error: 'Must be 18 or older'"
                },
                {
                    "id": "EP3",
                    "range": "> 65",
                    "description": "Above maximum age",
                    "representative_values": [66, 100, 150],  # Boundary + realistic + extreme
                    "expected_result": "Reject with error: 'Age exceeds maximum'"
                },
                {
                    "id": "EP4",
                    "range": "non-numeric",
                    "description": "Invalid data type",
                    "representative_values": ["abc", "", "12.5", "-25", None],
                    "expected_result": "Reject with error: 'Invalid age format'"
                }
            ]
        }
        
        # Test cases derived
        test_cases = []
        for category in partitions.values():
            for partition in category:
                for value in partition["representative_values"]:
                    test_cases.append({
                        "input": value,
                        "partition": partition["id"],
                        "expected": partition["expected_result"]
                    })
        
        return {
            "partitions": partitions,
            "minimum_test_cases": len(test_cases),
            "test_cases": test_cases
        }
    
    def analyze_loan_approval(self):
        """
        Multi-field equivalence partitioning
        Credit score: 300-850 (Valid), <300 or >850 (Invalid)
        Income: > $30,000 (Valid), <= $30,000 (Invalid)
        """
        # Input domains
        credit_score = {
            "valid": range(300, 851),
            "invalid_low": range(-999, 300),
            "invalid_high": range(851, 9999),
            "invalid_type": ["abc", None, ""]
        }
        
        income = {
            "valid": range(30001, 1000000),
            "invalid": range(0, 30001),
            "invalid_type": ["low", -5000, None]
        }
        
        # Number of partitions: 4 (credit) × 2 (income) = 8 valid combinations
        # Plus invalid combinations
        
        combinations = [
            {"credit": 700, "income": 50000, "expected": "Approved"},
            {"credit": 299, "income": 50000, "expected": "Rejected - Invalid credit"},
            {"credit": 700, "income": 25000, "expected": "Rejected - Income too low"},
            {"credit": "abc", "income": 50000, "expected": "Rejected - Invalid data"}
        ]
        
        return combinations
```

**Best Practices:**
1. Identify all input conditions and output conditions
2. Partition continuous ranges into valid/invalid classes
3. Consider data type partitions (numeric, string, null)
4. For multiple inputs, use **Weak Robust EP** (test one invalid at a time) or **Strong Robust EP** (test all combinations)

---

### **6.2.2 Boundary Value Analysis (BVA)**

**Concept:** Errors occur at boundaries between partitions. Test values at the edges (minimum, maximum, just inside, just outside) rather than arbitrary middle values.

**Theory:**
For a range $[min, max]$, test:
- $min - 1$ (just below)
- $min$ (on boundary)
- $min + 1$ (just above)
- $max - 1$ (just below)
- $max$ (on boundary)
- $max + 1$ (just above)

**Standard BVA (4 values per boundary):** $min, min+, max-, max$
**Robust BVA (6 values):** Adds $min-1, max+1$ (invalid boundaries)

```python
class BoundaryValueAnalysis:
    """
    Systematic boundary value testing implementation
    """
    
    def standard_bva(self, min_val, max_val):
        """
        Standard BVA: 4 values for single boundary
        For range [1, 100]: test 1, 2, 99, 100
        """
        return [min_val, min_val + 1, max_val - 1, max_val]
    
    def robust_bva(self, min_val, max_val):
        """
        Robust BVA: 6 values including invalid boundaries
        For range [1, 100]: test 0, 1, 2, 99, 100, 101
        """
        return [min_val - 1, min_val, min_val + 1, 
                max_val - 1, max_val, max_val + 1]
    
    def worst_case_bva(self, ranges):
        """
        For n inputs, test all combinations of boundary values
        5^n test cases (min-, min, min+, max-, max, max+)
        """
        import itertools
        
        all_values = []
        for min_val, max_val in ranges:
            values = [min_val-1, min_val, min_val+1, max_val-1, max_val, max_val+1]
            all_values.append(values)
        
        # Cartesian product
        combinations = list(itertools.product(*all_values))
        return combinations
    
    def practical_example_password_length(self):
        """
        Password must be 8-20 characters
        """
        min_len, max_len = 8, 20
        
        # Robust BVA test cases
        test_cases = [
            {
                "input": "A" * 7,  # 7 chars (min-1)
                "boundary": "Below minimum",
                "expected": "Error: Password must be at least 8 characters"
            },
            {
                "input": "A" * 8,  # 8 chars (min)
                "boundary": "Minimum boundary",
                "expected": "Accept"
            },
            {
                "input": "A" * 9,  # 9 chars (min+1)
                "boundary": "Just above minimum",
                "expected": "Accept"
            },
            {
                "input": "A" * 19, # 19 chars (max-1)
                "boundary": "Just below maximum",
                "expected": "Accept"
            },
            {
                "input": "A" * 20, # 20 chars (max)
                "boundary": "Maximum boundary",
                "expected": "Accept"
            },
            {
                "input": "A" * 21, # 21 chars (max+1)
                "boundary": "Above maximum",
                "expected": "Error: Password must not exceed 20 characters"
            }
        ]
        
        # Special boundary: Empty/null
        test_cases.extend([
            {
                "input": "",
                "boundary": "Empty string",
                "expected": "Error: Password required"
            },
            {
                "input": None,
                "boundary": "Null value",
                "expected": "Error: Password required"
            }
        ])
        
        return test_cases
    
    def two_variable_bva(self):
        """
        For two inputs X [1-100], Y [1-50]
        Standard: 4n+1 = 9 test cases
        Robust: 6n+1 = 13 test cases (including invalid combinations)
        """
        # Standard BVA (nominal values)
        X, Y = 50, 25  # Nominal middle values
        
        test_cases = [
            # On the boundary
            (1, Y),    # X min
            (100, Y),  # X max
            (X, 1),    # Y min
            (X, 50),   # Y max
            
            # Just inside
            (2, Y),    # X min+
            (99, Y),   # X max-
            (X, 2),    # Y min+
            (X, 49),   # Y max-
            
            # Nominal
            (X, Y)     # Center
        ]
        
        return test_cases
```

**Boundary Types to Consider:**
1. **Numeric ranges:** Min, max values
2. **Array/String lengths:** First, last elements; empty; full
3. **Time/Date:** Start/end of day, month, year; leap years
4. **File systems:** Empty file, maximum size, disk full
5. **Loops:** Zero iterations, one iteration, maximum iterations, maximum+1

---

### **6.2.3 Decision Table Testing**

**Concept:** Used when system behavior depends on combinations of conditions (rules). Creates a matrix of conditions vs. actions to ensure all combinations are tested.

**Structure:**
- **Conditions (Inputs):** Stated as True/False or Yes/No
- **Actions (Outputs):** What the system should do
- **Rules:** Columns representing unique combinations

**Coverage:**
- **Complete/Exhaustive:** All possible combinations ($2^n$ for n binary conditions)
- **Collapsed/Reduced:** Merge rules where actions are identical regardless of one condition (marked with "-")

```python
class DecisionTableTesting:
    """
    Decision table design and optimization
    """
    
    def credit_card_approval_example(self):
        """
        Credit card approval based on:
        C1: Credit score > 700
        C2: Income > $50k
        C3: No existing defaults
        """
        
        # Full decision table (2^3 = 8 rules)
        full_table = {
            "conditions": {
                "C1": "Credit score > 700",
                "C2": "Income > $50k", 
                "C3": "No existing defaults"
            },
            "rules": [
                # C1, C2, C3 | Action
                {"id": 1, "C1": True,  "C2": True,  "C3": True,  "Action": "Approve with high limit"},
                {"id": 2, "C1": True,  "C2": True,  "C3": False, "Action": "Reject (default history)"},
                {"id": 3, "C1": True,  "C2": False, "C3": True,  "Action": "Approve with low limit"},
                {"id": 4, "C1": True,  "C2": False, "C3": False, "Action": "Reject (default history)"},
                {"id": 5, "C1": False, "C2": True,  "C3": True,  "Action": "Manual review"},
                {"id": 6, "C1": False, "C2": True,  "C3": False, "Action": "Reject (default history)"},
                {"id": 7, "C1": False, "C2": False, "C3": True,  "Action": "Reject (low score)"},
                {"id": 8, "C1": False, "C2": False, "C3": False, "Action": "Reject (multiple factors)"}
            ]
        }
        
        # Optimization: Notice rules 2, 4, 6 all result in "Reject" due to C3=False
        # We can collapse these
        
        collapsed_table = {
            "rules": [
                {"C1": True,  "C2": True,  "C3": True,  "Action": "Approve with high limit"},
                {"C1": True,  "C2": False, "C3": True,  "Action": "Approve with low limit"},
                {"C1": False, "C2": True,  "C3": True,  "Action": "Manual review"},
                {"C1": "-",   "C2": "-",   "C3": False, "Action": "Reject (default history)"},  # Collapsed
                {"C1": False, "C2": False, "C3": True,  "Action": "Reject (low score)"}
            ]
        }
        
        return {
            "full_table": full_table,
            "collapsed_table": collapsed_table,
            "test_cases": len(collapsed_table["rules"])  # 5 instead of 8
        }
    
    def generate_test_cases_from_table(self, decision_table):
        """
        Convert decision table rows to test cases
        """
        test_cases = []
        
        for rule in decision_table["rules"]:
            test_case = {
                "id": f"DT_{rule['id']}",
                "preconditions": [],
                "inputs": {},
                "expected_result": rule["Action"]
            }
            
            # Map conditions to test data
            for cond_id, value in rule.items():
                if cond_id not in ["id", "Action"]:
                    if value == True:
                        test_case["inputs"][cond_id] = self._get_true_value(cond_id)
                    elif value == False:
                        test_case["inputs"][cond_id] = self._get_false_value(cond_id)
                    # "-" means don't care, any valid value
            
            test_cases.append(test_case)
        
        return test_cases
    
    def _get_true_value(self, condition):
        """Map condition to realistic test data"""
        mappings = {
            "C1": 750,  # Credit score > 700
            "C2": 60000,  # Income > 50k
            "C3": True  # No defaults
        }
        return mappings.get(condition, True)
    
    def _get_false_value(self, condition):
        mappings = {
            "C1": 650,
            "C2": 40000,
            "C3": False
        }
        return mappings.get(condition, False)
```

**When to Use Decision Tables:**
- Business rules with multiple conditions
- Complex validation logic
- Workflow approvals
- Pricing calculations with multiple factors
- Configuration testing

---

### **6.2.4 State Transition Testing**

**Concept:** Used when system behavior depends on current state and input events (finite state machines). Tests valid transitions, invalid transitions, and state entry/exit.

**Components:**
- **States:** Conditions of the system (e.g., Idle, Processing, Complete)
- **Transitions:** Movement between states triggered by events
- **Events:** Inputs or actions causing transitions
- **Actions:** Activities performed during transitions or in states

**Coverage Levels:**
1. **0-Switch (State Coverage):** Visit every state at least once
2. **1-Switch (Transition Coverage):** Traverse every transition at least once
3. **2-Switch (Transition Pair Coverage):** Test consecutive transition pairs

```python
class StateTransitionTesting:
    """
    State machine testing for workflow-driven applications
    """
    
    def atm_state_machine(self):
        """
        ATM State Transition example
        States: Idle -> CardInserted -> PINEntered -> Transaction -> Complete
        """
        
        states = {
            "S1_Idle": "Waiting for card",
            "S2_CardInserted": "Card validated, waiting for PIN",
            "S3_PINEntered": "PIN validated, waiting for transaction choice",
            "S4_Transaction": "Processing transaction",
            "S5_Complete": "Transaction complete, ejecting card"
        }
        
        transitions = [
            # (From_State, Event, To_State, Action)
            ("S1_Idle", "Insert Valid Card", "S2_CardInserted", "Validate card"),
            ("S2_CardInserted", "Enter Valid PIN", "S3_PINEntered", "Validate PIN"),
            ("S3_PINEntered", "Select Withdrawal", "S4_Transaction", "Check balance"),
            ("S4_Transaction", "Valid Amount", "S5_Complete", "Dispense cash"),
            ("S4_Transaction", "Invalid Amount", "S3_PINEntered", "Show error"),
            ("S3_PINEntered", "Select Balance", "S5_Complete", "Print receipt"),
            ("S2_CardInserted", "Enter Invalid PIN", "S2_CardInserted", "Increment counter"),
            ("S2_CardInserted", "3 Invalid PINs", "S1_Idle", "Retain card"),
            ("S5_Complete", "Card Ejected", "S1_Idle", "Reset")
        ]
        
        # 0-Switch Coverage: Visit all states (5 test cases minimum)
        zero_switch = ["S1", "S2", "S3", "S4", "S5"]
        
        # 1-Switch Coverage: Cover all transitions (9 transitions = 9 test cases)
        one_switch_paths = [
            ["S1", "Insert Valid Card", "S2"],
            ["S2", "Enter Valid PIN", "S3"],
            ["S3", "Select Withdrawal", "S4"],
            ["S4", "Valid Amount", "S5"],
            ["S4", "Invalid Amount", "S3"],  # Loop back
            ["S3", "Select Balance", "S5"],
            ["S2", "Enter Invalid PIN", "S2"],  # Self-loop
            ["S2", "3 Invalid PINs", "S1"],
            ["S5", "Card Ejected", "S1"]
        ]
        
        # 2-Switch Coverage: Pairs of transitions (N-1 transitions in sequence)
        two_switch_paths = [
            # Happy path
            ["S1", "Insert Valid Card", "S2", "Enter Valid PIN", "S3", "Select Withdrawal", "S4", "Valid Amount", "S5"],
            # Invalid PIN then valid
            ["S2", "Enter Invalid PIN", "S2", "Enter Valid PIN", "S3"],
            # Withdrawal with invalid amount then valid
            ["S3", "Select Withdrawal", "S4", "Invalid Amount", "S3", "Select Withdrawal", "S4", "Valid Amount", "S5"]
        ]
        
        return {
            "states": states,
            "transitions": transitions,
            "coverage_0_switch": zero_switch,
            "coverage_1_switch": one_switch_paths,
            "coverage_2_switch": two_switch_paths
        }
    
    def invalid_transitions(self, state_machine):
        """
        Test what happens when invalid events occur in states
        """
        invalid_tests = [
            {
                "current_state": "S1_Idle",
                "invalid_event": "Enter PIN",
                "expected": "Error: No card inserted",
                "system_should": "Remain in S1"
            },
            {
                "current_state": "S3_PINEntered",
                "invalid_event": "Insert Card",
                "expected": "Error: Card already inserted",
                "system_should": "Remain in S3"
            },
            {
                "current_state": "S5_Complete",
                "invalid_event": "Select Withdrawal",
                "expected": "Error: Session ended",
                "system_should": "Remain in S5 or transition to S1"
            }
        ]
        return invalid_tests
    
    def state_table_representation(self):
        """
        Alternative to diagram: State table
        Rows = Current State, Columns = Event
        """
        table = {
            "events": ["Insert Card", "Enter PIN", "Select Withdraw", "Valid Amount", "Cancel"],
            "transitions": {
                "Idle": ["CardInserted", "Error", "Error", "Error", "Error"],
                "CardInserted": ["Error", "PINEntered", "Error", "Error", "Idle"],
                "PINEntered": ["Error", "Error", "Transaction", "Error", "Idle"],
                "Transaction": ["Error", "Error", "Error", "Complete", "Idle"],
                "Complete": ["Error", "Error", "Error", "Error", "Idle"]
            }
        }
        return table
```

**Best Practices:**
- Create state diagram first (visual representation)
- Identify trap states (dead ends) and initial/final states
- Test self-transitions (loops)
- Test invalid events in each state (robustness)
- Consider guard conditions (transitions only if condition true)

---

### **6.2.5 Use Case Testing**

**Concept:** Derives test cases from use cases (user scenarios) to validate end-to-end workflows. Each use case typically yields one or more test cases covering basic flow and alternative flows.

**Structure:**
- **Actor:** User or system initiating action
- **Precondition:** System state before use case
- **Basic Flow:** Normal, expected path (sunny day scenario)
- **Alternative Flows:** Variations, exceptions, edge cases
- **Postcondition:** System state after successful completion

```python
class UseCaseTesting:
    """
    Test design from use case specifications
    """
    
    def e_commerce_purchase_use_case(self):
        """
        Use Case: Purchase Product
        Actor: Registered Customer
        """
        
        use_case = {
            "id": "UC_001",
            "name": "Purchase Product",
            "actor": "Registered Customer",
            "precondition": "Customer logged in, product in cart",
            
            "basic_flow": [
                "1. Customer views shopping cart",
                "2. System displays cart items and total",
                "3. Customer clicks 'Checkout'",
                "4. System displays shipping options",
                "5. Customer selects shipping method",
                "6. System calculates total with shipping",
                "7. Customer enters payment information",
                "8. System validates payment",
                "9. System processes payment",
                "10. System displays order confirmation",
                "11. System sends confirmation email"
            ],
            
            "alternative_flows": {
                "A1": {
                    "name": "Empty Cart",
                    "trigger": "Step 1: Cart is empty",
                    "action": "System displays 'Cart is empty' message",
                    "return_to": "End use case"
                },
                "A2": {
                    "name": "Invalid Shipping Address",
                    "trigger": "Step 4: Address validation fails",
                    "action": "System prompts for valid address",
                    "return_to": "Step 4"
                },
                "A3": {
                    "name": "Payment Declined",
                    "trigger": "Step 8: Bank declines payment",
                    "action": "System displays error, allows retry or different payment",
                    "return_to": "Step 7"
                },
                "A4": {
                    "name": "Session Timeout",
                    "trigger": "Any step: Session expires",
                    "action": "System saves cart, redirects to login",
                    "return_to": "Step 1 after login"
                }
            },
            
            "postcondition": "Order created, inventory reduced, email sent"
        }
        
        # Generate test cases
        test_cases = [
            {
                "id": "TC_UC001_01",
                "name": "Basic Flow - Successful Purchase",
                "path": "1-2-3-4-5-6-7-8-9-10-11",
                "data": {
                    "cart_items": [{"sku": "ITEM001", "qty": 2}],
                    "shipping": "Standard",
                    "payment": "Valid Visa ending 4242"
                },
                "expected": "Order confirmed, email received"
            },
            {
                "id": "TC_UC001_02",
                "name": "Alternative Flow - Empty Cart",
                "path": "1-A1",
                "data": {"cart_items": []},
                "expected": "Empty cart message displayed"
            },
            {
                "id": "TC_UC001_03",
                "name": "Alternative Flow - Payment Declined then Retry",
                "path": "1-2-3-4-5-6-7-8-A3-7-8-9-10-11",
                "data": {
                    "first_payment": "Declined card",
                    "second_payment": "Valid card"
                },
                "expected": "Initial decline shown, retry succeeds"
            }
        ]
        
        return {
            "use_case": use_case,
            "test_cases": test_cases,
            "coverage": "Basic flow + All alternative flows"
        }
    
    def use_case_coverage_criteria(self):
        """
        Coverage levels for use case testing
        """
        return {
            "scenario_coverage": {
                "level_1": "Test basic flow only (sunny day)",
                "level_2": "Test basic flow + each alternative flow separately",
                "level_3": "Test combinations of alternative flows",
                "level_4": "Test all paths (often impractical for complex use cases)"
            },
            "step_coverage": "Every step in basic flow executed at least once",
            "actor_coverage": "Test with different actor types (if applicable)"
        }
```

---

### **6.2.6 Error Guessing**

**Concept:** Experience-based technique where testers use intuition and past experience to anticipate where defects might occur. Complements systematic techniques but does not replace them.

**Common Error-Prone Areas:**
- Boundary conditions (off-by-one errors)
- Null/empty inputs
- Division by zero
- File handling (permissions, missing files, disk full)
- Concurrency (race conditions, deadlocks)
- Date/time handling (time zones, leap years, DST)
- Internationalization (character encoding, currencies)

```python
class ErrorGuessing:
    """
    Experience-based test design
    """
    
    def common_error_patterns(self):
        """
        Patterns where developers commonly make mistakes
        """
        patterns = {
            "input_validation": [
                "Empty string vs None vs 'null' string",
                "Leading/trailing spaces (trimming)",
                "Unicode characters (emojis, accents)",
                "Maximum length + 1",
                "SQL injection attempts ('; DROP TABLE--)",
                "XSS attempts (<script>alert('xss')</script>)"
            ],
            
            "arithmetic": [
                "Division by zero",
                "Integer overflow (2^31, 2^63)",
                "Floating point precision (0.1 + 0.2 != 0.3)",
                "Negative numbers where only positive expected"
            ],
            
            "date_time": [
                "February 29 on non-leap year",
                "Daylight saving time transitions (missing hour)",
                "Time zone conversions (UTC vs local)",
                "Date parsing (DD/MM/YYYY vs MM/DD/YYYY)",
                "Epoch time 0 (Jan 1 1970)"
            ],
            
            "file_system": [
                "File not found",
                "File locked by another process",
                "Path traversal (../../../etc/passwd)",
                "Zero-byte files",
                "Files > 2GB (32-bit limits)",
                "Special characters in filenames"
            ],
            
            "concurrency": [
                "Double-click submit button (duplicate requests)",
                "Rapid successive operations",
                "Logout while operation in progress",
                "Browser back button after form submit"
            ],
            
            "network": [
                "Connection timeout",
                "Connection dropped mid-transaction",
                "Slow network (3G simulation)",
                "Offline mode"
            ]
        }
        return patterns
    
    def exploratory_charters(self):
        """
        Time-boxed exploratory testing based on error guessing
        """
        charters = [
            {
                "title": "Input Validation Attack",
                "duration": "90 minutes",
                "focus": "Find input validation weaknesses",
                "heuristics": ["Goldilocks (too small, too big, just right)", 
                              "CRUD (Create invalid, Read invalid, etc.)"],
                "data": "Prepare fuzzing data: special chars, long strings, format strings"
            },
            {
                "title": "Race Condition Hunt",
                "duration": "60 minutes",
                "focus": "Concurrent operations",
                "heuristics": ["Multiple tabs", "Double submission", "Fast clicks"],
                "data": "Setup: Two browsers, automation script for rapid requests"
            }
        ]
        return charters
```

---

## **6.3 White Box Testing Techniques**

White box (structure-based) techniques derive test cases from code structure, logic, and internal paths. These require programming knowledge and access to source code.

### **6.3.1 Statement Coverage**

**Concept:** Every executable statement in the code is executed at least once.

**Formula:**
$$ \text{Statement Coverage} = \frac{\text{Number of statements executed}}{\text{Total number of executable statements}} \times 100\% $$

**Limitation:** 100% statement coverage does not guarantee all decisions are tested (e.g., missing else branches).

```python
class StatementCoverage:
    """
    Analyzing code for statement coverage
    """
    
    def example_code(self):
        """
        Code under test:
        
        def calculate_discount(price, is_member):
            discount = 0
            if is_member:           # Statement 1
                discount = 0.1      # Statement 2
            final_price = price * (1 - discount)  # Statement 3
            return final_price      # Statement 4
        """
        
        # Total executable statements: 4
        
        test_cases = [
            {
                "input": {"price": 100, "is_member": True},
                "executed_statements": [1, 2, 3, 4],  # All statements
                "coverage": "100%"
            },
            {
                "input": {"price": 100, "is_member": False},
                "executed_statements": [1, 3, 4],  # Statement 2 skipped
                "coverage": "75% (3/4)"
            }
        ]
        
        # To achieve 100% statement coverage, we need both test cases
        
        return {
            "minimum_test_cases": 2,
            "test_cases": test_cases,
            "weakness": "Statement coverage does not require testing the else path explicitly if the if path exists"
        }
    
    def calculate_coverage(self, code_ast, executed_lines):
        """
        Calculate statement coverage from execution trace
        """
        total_statements = len([node for node in code_ast if self.is_executable(node)])
        executed = len(set(executed_lines))
        
        coverage = (executed / total_statements) * 100
        return {
            "percentage": coverage,
            "executed": executed,
            "total": total_statements,
            "missing": total_statements - executed
        }
```

---

### **6.3.2 Branch Coverage (Decision Coverage)**

**Concept:** Every decision (if, while, for, switch) evaluates to both True and False at least once. Every branch is taken.

**Formula:**
$$ \text{Branch Coverage} = \frac{\text{Number of branches executed}}{\text{Total number of branches}} \times 100\% $$

**Stronger than statement coverage:** Requires testing both outcomes of decisions.

```python
class BranchCoverage:
    """
    Decision/branch coverage analysis
    """
    
    def branch_analysis(self):
        """
        Code:
        if (age > 18 and has_license):
            rent_car()    # Branch A
        else:
            deny()        # Branch B
        
        Decisions:
        - age > 18: True/False
        - has_license: True/False (only evaluated if age > 18 is True in many languages)
        """
        
        # Decision points: 2 (age check, license check)
        # Total branches: 4 (TT, TF, FT, FF) - though TF may be unreachable in short-circuit evaluation
        
        test_cases = [
            {
                "name": "Eligible adult with license",
                "input": {"age": 25, "has_license": True},
                "branches_covered": ["age>18:T", "has_license:T", "rent_car"],
                "decision_outcomes": [(True, True)]
            },
            {
                "name": "Eligible adult no license",
                "input": {"age": 25, "has_license": False},
                "branches_covered": ["age>18:T", "has_license:F", "deny"],
                "decision_outcomes": [(True, False)]
            },
            {
                "name": "Minor",
                "input": {"age": 16, "has_license": True},  # License irrelevant
                "branches_covered": ["age>18:F", "deny"],
                "decision_outcomes": [(False, None)]  # Short-circuit
            }
        ]
        
        # Coverage achieved:
        # Decision 1 (age>18): T and F covered ✓
        # Decision 2 (has_license): T and F covered ✓
        
        return {
            "branch_coverage": "100%",
            "test_cases_required": 3,
            "note": "Branch coverage ensures both outcomes of each decision are tested"
        }
    
    def branch_vs_statement(self):
        """
        Demonstrating difference between statement and branch coverage
        """
        code = """
        def check_value(x):
            if x > 0:           # Decision
                return "positive"   # Statement 1
            return "non-positive"   # Statement 2 (implicit else)
        """
        
        # Statement coverage: 1 test case (x=1 covers both statements)
        # Actually: x=1 executes if and return "positive"
        # x=-1 executes if (false) and return "non-positive"
        # So we need 2 tests for 100% statement coverage too in this case
        
        # Better example:
        code2 = """
        def process(x):
            if x > 0:
                do_something()
            do_always()  # Outside if
        """
        
        # Statement coverage: x=1 covers all 3 statements
        # Branch coverage: needs x=1 (true branch) and x=0 (false branch)
        
        return {
            "conclusion": "Branch coverage is stronger than statement coverage"
        }
```

---

### **6.3.3 Path Coverage**

**Concept:** Every possible route through the code (combination of branches) is executed at least once.

**Complexity:** For n independent decisions, there are $2^n$ paths. Impractical for complex code but ideal for critical safety systems.

**Cyclomatic Complexity:**
$$V(G) = E - N + 2P$$
Where $E$ = edges, $N$ = nodes, $P$ = connected components (usually 1).

Number of independent paths = $V(G)$.

```python
class PathCoverage:
    """
    Path testing and cyclomatic complexity
    """
    
    def cyclomatic_complexity(self, code_structure):
        """
        Calculate McCabe's Cyclomatic Complexity
        """
        # Method 1: V(G) = Number of predicate nodes (decisions) + 1
        decisions = code_structure["decisions"]  # if, while, for, case
        complexity = decisions + 1
        
        # Method 2: V(G) = Edges - Nodes + 2
        edges = code_structure["edges"]
        nodes = code_structure["nodes"]
        complexity_check = edges - nodes + 2
        
        return {
            "cyclomatic_complexity": complexity,
            "independent_paths": complexity,  # Number of paths to test
            "interpretation": {
                "1-10": "Simple, low risk",
                "11-20": "Moderate risk",
                "21-50": "High risk, testing challenging",
                "50+": "Very high risk, untestable, refactor needed"
            }
        }
    
    def path_analysis_example(self):
        """
        Code with nested decisions:
        
        def process_order(amount, is_vip, in_stock):
            if amount > 1000:               # D1
                if is_vip:                  # D2
                    discount = 0.2          # Path 1
                else:
                    discount = 0.1          # Path 2
            else:
                discount = 0                # Path 3
            
            if in_stock:                    # D3
                ship_order()                # Path A
            else:
                backorder()                 # Path B
        
        Total paths: 3 × 2 = 6 independent paths
        """
        
        paths = [
            "Path 1: amount>1000(T) → is_vip(T) → in_stock(T)",
            "Path 2: amount>1000(T) → is_vip(T) → in_stock(F)",
            "Path 3: amount>1000(T) → is_vip(F) → in_stock(T)",
            "Path 4: amount>1000(T) → is_vip(F) → in_stock(F)",
            "Path 5: amount>1000(F) → in_stock(T)",
            "Path 6: amount>1000(F) → in_stock(F)"
        ]
        
        return {
            "total_paths": 6,
            "cyclomatic_complexity": 4,  # 3 decisions + 1
            "note": "100% path coverage often impractical; aim for basis path coverage (cyclomatic complexity number of paths)"
        }
    
    def basis_path_testing(self):
        """
        Test a basis set of paths that cover all edges
        """
        # Instead of all 2^n paths, test V(G) paths that form a basis
        # Guarantees statement and branch coverage, but not all combinations
        
        basis_set = [
            "Happy path: All true conditions",
            "First decision false, rest true",
            "First true, second false, third true",
            "First two true, third false"
        ]
        
        return basis_set
```

---

### **6.3.4 Condition Coverage**

**Concept:** Every Boolean sub-expression (condition) evaluates to both True and False.

**Types:**
1. **Condition Coverage:** Each atomic condition is T/F (doesn't guarantee decision coverage)
2. **Decision/Condition Coverage:** Both decision and condition coverage
3. **Modified Condition/Decision Coverage (MC/DC):** Each condition independently affects the decision outcome (required for DO-178C avionics software)

```python
class ConditionCoverage:
    """
    Boolean condition testing
    """
    
    def compound_condition_example(self):
        """
        if (age > 18 and credit_score > 700):
            approve()
        
        Atomic conditions:
        C1: age > 18
        C2: credit_score > 700
        """
        
        # Simple Condition Coverage (each condition T/F):
        test_cases_simple = [
            {"age": 25, "credit": 750, "C1": True, "C2": True},   # Both T
            {"age": 16, "credit": 600, "C1": False, "C2": False}  # Both F
        ]
        # Problem: Decision always True in first, False in second
        # We never test decision outcome changes
        
        # Decision/Condition Coverage (each condition T/F + Decision T/F):
        test_cases_dcc = [
            {"age": 25, "credit": 750, "C1": True, "C2": True, "Decision": True},
            {"age": 25, "credit": 600, "C1": True, "C2": False, "Decision": False},
            {"age": 16, "credit": 750, "C1": False, "C2": True, "Decision": False}
        ]
        
        return {
            "simple_condition": test_cases_simple,
            "decision_condition": test_cases_dcc,
            "coverage_achieved": "Each condition T/F, Decision T/F"
        }
    
    def mcdc_coverage(self):
        """
        Modified Condition/Decision Coverage (MC/DC)
        Required for safety-critical systems (DO-178B/C Level A)
        
        Each condition must independently affect the decision outcome
        """
        
        # For condition (A and B and C):
        # Need pairs where only one condition changes, decision changes
        
        mcdc_pairs = [
            # Changing A from F to T changes decision from F to T
            {"A": False, "B": True, "C": True, "Decision": False, "changing": "A"},
            {"A": True,  "B": True, "C": True, "Decision": True,  "changing": "A"},
            
            # Changing B from F to T changes decision (when A=T, C=T)
            {"A": True, "B": False, "C": True, "Decision": False, "changing": "B"},
            {"A": True, "B": True,  "C": True, "Decision": True,  "changing": "B"},
            
            # Changing C from F to T changes decision (when A=T, B=T)
            {"A": True, "B": True, "C": False, "Decision": False, "changing": "C"},
            {"A": True, "B": True, "C": True,  "Decision": True,  "changing": "C"}
        ]
        
        # Minimum test cases for MC/DC = N + 1 (where N = number of conditions)
        # For 3 conditions, minimum 4 test cases (not 6 as shown above, optimized pairs exist)
        
        return {
            "minimum_test_cases": "N + 1 where N = number of independent conditions",
            "requirement": "Each condition must independently affect the decision outcome",
            "application": "Aviation (DO-178C), Nuclear, Medical devices"
        }
```

---

## **6.4 Grey Box Testing**

**Concept:** Combines black box and white box approaches. Tester has partial knowledge of internal structure (e.g., database schema, API contracts) but tests from user perspective.

**Techniques:**
- **API Testing:** Knowing endpoints and schemas but testing inputs/outputs
- **Database Testing:** Verifying data persistence without knowing stored procedures
- **Pattern Testing:** Knowing architecture patterns to predict failure modes

```python
class GreyBoxTesting:
    """
    Hybrid testing approach
    """
    
    def api_schema_testing(self):
        """
        Testing API with knowledge of schema but as external consumer
        """
        # Known: OpenAPI/Swagger schema
        schema = {
            "endpoint": "/api/users",
            "method": "POST",
            "request_schema": {
                "name": {"type": "string", "maxLength": 50},
                "email": {"type": "string", "format": "email"},
                "age": {"type": "integer", "minimum": 18, "maximum": 120}
            },
            "response_schema": {
                "201": {"id": "integer", "created_at": "datetime"},
                "400": {"error": "string", "code": "integer"}
            }
        }
        
        # Test cases combine black box (invalid inputs) with schema knowledge (boundaries)
        test_cases = [
            # Valid (happy path)
            {"name": "John", "email": "john@example.com", "age": 30, "expected": 201},
            
            # Boundary (schema maxLength)
            {"name": "A" * 50, "email": "a@b.co", "age": 18, "expected": 201},
            
            # Invalid (beyond schema)
            {"name": "A" * 51, "email": "john@example.com", "age": 30, "expected": 400},
            
            # SQL injection (black box security)
            {"name": "'; DROP TABLE users;--", "email": "test@test.com", "age": 30, "expected": 400}
        ]
        
        return test_cases
    
    def database_verification(self):
        """
        Verify UI actions result in correct database state
        """
        test_case = {
            "action": "User registers via web form",
            "input": {"username": "testuser", "email": "test@example.com"},
            
            # Grey box: We know database structure but test through UI
            "database_checks": [
                "SELECT COUNT(*) FROM users WHERE email='test@example.com' = 1",
                "SELECT email_verified FROM users WHERE username='testuser' = False",
                "SELECT created_at is not NULL FROM users WHERE username='testuser'"
            ],
            
            "white_box_avoided": "Do not test: Trigger logic, stored procedure implementation, indexing"
        }
        
        return test_case
```

---

## **6.5 Selecting Test Design Techniques**

Choosing the right technique depends on risk, time, knowledge, and test level.

```python
class TechniqueSelection:
    """
    Guidance for selecting appropriate techniques
    """
    
    def selection_criteria(self):
        return {
            "risk_level_high": {
                "techniques": ["Decision Table", "State Transition", "MC/DC", "Path Testing"],
                "rationale": "Thorough testing of complex logic and states"
            },
            "risk_level_medium": {
                "techniques": ["Equivalence Partitioning", "Boundary Value", "Branch Coverage"],
                "rationale": "Good coverage with manageable effort"
            },
            "risk_level_low": {
                "techniques": ["Error Guessing", "Statement Coverage", "Smoke Testing"],
                "rationale": "Quick validation sufficient"
            },
            
            "test_level_unit": {
                "white_box": ["Statement", "Branch", "Path", "MC/DC"],
                "black_box": ["Equivalence", "Boundary"]
            },
            "test_level_integration": {
                "grey_box": ["API Schema Testing", "Data Flow"],
                "black_box": ["Decision Table", "State Transition"]
            },
            "test_level_system": {
                "black_box": ["Use Case", "Decision Table", "State Transition", "EP/BVA"],
                "experience": ["Error Guessing", "Exploratory"]
            },
            "test_level_acceptance": {
                "black_box": ["Use Case", "Scenario Testing"],
                "business_focus": ["User Story Testing"]
            }
        }
    
    def combination_strategy(self):
        """
        Using multiple techniques together
        """
        strategy = {
            "minimum_recommended": [
                "Equivalence Partitioning (validates partitions)",
                "Boundary Value Analysis (catches off-by-one errors)",
                "Error Guessing (experience-based edge cases)"
            ],
            "thorough_combination": [
                "EP/BVA for input validation",
                "Decision Tables for business rules",
                "State Transition for workflows",
                "Branch Coverage for critical code"
            ]
        }
        return strategy
```

---

## **Chapter Summary**

This chapter equipped you with systematic techniques for designing effective test cases that maximize defect detection while minimizing redundancy.

**Black Box Techniques (Specification-Based):**
- **Equivalence Partitioning:** Divides inputs into classes, testing one representative per class. Reduces test cases from infinite to manageable while maintaining coverage.
- **Boundary Value Analysis:** Tests edges of partitions where errors cluster. Essential for catching off-by-one errors and boundary violations.
- **Decision Tables:** Systematically tests combinations of conditions and actions. Ideal for complex business rules with multiple interdependent factors.
- **State Transition:** Models system behavior as states and transitions. Critical for testing workflows, protocols, and UI navigation.
- **Use Case Testing:** Validates end-to-end user scenarios including alternative flows and exceptions. Ensures business requirements are met.
- **Error Guessing:** Leverages experience and heuristics to find defects in historically error-prone areas.

**White Box Techniques (Structure-Based):**
- **Statement Coverage:** Baseline coverage ensuring every line executes. Weak but necessary minimum.
- **Branch/Decision Coverage:** Tests both True and False outcomes of every decision. Stronger than statement coverage.
- **Path Coverage:** Tests all paths through code. Ideal for safety-critical systems but often impractical due to combinatorial explosion.
- **Condition Coverage:** Tests atomic Boolean conditions. MC/DC variant required for aviation and medical devices.

**Grey Box Testing:** Combines specification knowledge with partial internal visibility (API schemas, database structure) for effective integration testing.

**Selection Strategy:**
- Match technique to risk (high risk = thorough techniques like decision tables and path testing)
- Match technique to test level (unit = white box, system = black box)
- Combine techniques (EP + BVA + Error Guessing is a minimum standard)
- Consider constraints (time, tools, skills)

**Standards Alignment:**
- **ISTQB Foundation:** All techniques align with ISTQB syllabi
- **IEEE 1012:** Verification and validation standards
- **DO-178C:** MC/DC for safety-critical avionics

Mastering these techniques transforms test design from ad-hoc guessing into systematic engineering, ensuring that every test case provides maximum value in the quest for software quality.

---

## **📖 Next Chapter: Chapter 7 - Test Case Development**

Now that you understand the techniques for deriving test cases, **Chapter 7: Test Case Development** will guide you through the practical implementation of writing, organizing, and maintaining test cases for long-term value.

In **Chapter 7**, you will learn:

- **Test Case Authoring Best Practices:** Writing clear, unambiguous, executable test cases that any team member can run
- **Test Case Templates:** Industry-standard formats (IEEE 829) and tool-specific implementations (TestRail, Zephyr, Excel)
- **Test Data Management:** Strategies for creating, masking, subsetting, and maintaining test data
- **Test Case Prioritization:** Techniques for ranking tests by risk, business value, and failure history
- **Traceability Implementation:** Linking test cases to requirements, code, and defects for impact analysis
- **Maintenance Strategies:** Keeping test cases current as applications evolve (version control, review cycles)
- **Test Case Metrics:** Measuring the quality of your test cases (defect-finding efficiency, maintenance cost)

You will move from knowing *how* to design tests to *implementing* a sustainable test case library that serves your organization through multiple release cycles.

**Continue to Chapter 7 to master the art and science of professional test case development!**